# Fonctions permettant la reconnaissance des couronnes

## Récupération et Affichage des images

In [ ]:
def recupere(fichier):
    liste = []
    for img_name in os.listdir(fichier):
        if img_name.endswith('.jpeg') or img_name.endswith('.jpg'):
            liste.append(Image.open(os.path.join(fichier, img_name)))
    return liste

## Contour des noyaux
Max_X et Max-Y sont les coordonnées x et y des points du contour.
pixels_intensity x et y sont l'intensité et les coordonnées de tous les points De toutes les normales.

In [ ]:
def analyse(liste, normale_length, start_intensite):
    for image_liste in range(0, len(liste), 2):
        
        # Dans la liste, il faut toujours que l'image bleu soit après l'image verte
        num_image_bleue = image_liste + 1
        num_image_verte = image_liste
        image_bleue = np.array(liste[num_image_bleue])
        image_verte = np.array(liste[num_image_verte])
        
        # On fait le pré-traitement de l'image et on récupère les contours convexes
        contours, convexe_contoured_image, contours_convexe = contour_binary_fonction(image_bleue, num_image_bleue)
        
        num_contour = 1
        for contour in contours_convexe:
            # On ajoute des points dans le contour pour avoir plus de normales si nécessaire
            # et on trace le contour et recupere les cood x et y dans Max_X Max_Y
            points_contour, Max_X, Max_Y, contoured_image_points = points_image_fonction(contour, image_bleue)
            convexe_contour_image = image_bleue.copy()
            cv2.drawContours(convexe_contour_image, [contour], -1, (255, 255, 0), 2)

            colors = cm.hot(np.linspace(0, 1, len(points_contour)))

            # Pour chaque points, on trace la tangente, qui sera la normale
            normales = normale_fonction(points_contour, Max_X, Max_Y, normale_length, colors)

            # On récupère l'intensité des pixels et trace l'image avec les contours
            pixels_intensity_perception, pixels_x, pixels_y, contoured_image_normales = all_intensity_fonction(points_contour, Max_X, Max_Y, normale_length, start_intensite, normales, image_verte, num_image_verte, colors)

            # On récupère la moyenne et écart-type 
            moyenne, std = mean_std_fonction(pixels_intensity_perception)

            # POUR EXPLICATIONS : TRACE DE LA PREMIERE NORMALE
#             plt.figure(figsize=(5,4))
#             plt.axvspan(start_intensite, -15, facecolor='#e3e3e3ff', alpha=1) # Rectangle gris noyau
#             plt.axvspan(-15, 10, facecolor='#dffbdfff', alpha=1) # Rectangle vert PC
#             plt.axvspan(10, normale_length, facecolor='#e0e5f332', alpha=0.2) #Rectangle blanc cytoplasme
#             abscisse = np.linspace(start_intensite, normale_length, pixels_intensity_perception.shape[1])

#             masque = pixels_intensity_perception[0, :] > 0
#             abscisse_positif = abscisse[masque]
#             intensity_positif = pixels_intensity_perception[0, :][masque]
#             plt.plot(abscisse_positif, intensity_positif, color=colors[0], alpha=0.7, label='Normale 1')
#             plt.title(f'Pixel intensity as a function of distance', fontsize=18)
#             plt.xlabel('Distance in pixels', fontsize=18)
#             plt.ylabel('Pixel intensity', fontsize=18)
#             plt.legend(fontsize=14)

#             plt.tight_layout()
#             plt.show()

            # PLOT DU GRAPH COMPLET
            plt.figure(figsize=(9,6))
            plt.axvspan(start_intensite, -15, facecolor='#e3e3e3ff', alpha=1) # Rectangle gris noyau
            plt.axvspan(-15, 10, facecolor='#dffbdfff', alpha=1) # Rectangle vert PC
            plt.axvspan(10, normale_length, facecolor='#e0e5f332', alpha=0.2) #Rectangle blanc cytoplasme
            abscisse = np.linspace(start_intensite, normale_length, pixels_intensity_perception.shape[1])

            for i in range(pixels_intensity_perception.shape[0]):
                masque = pixels_intensity_perception[i, :] > 0
                abscisse_positif = abscisse[masque]
                intensity_positif = pixels_intensity_perception[i, :][masque]

                plt.plot(abscisse_positif, intensity_positif, color=colors[i], alpha=0.1)

            plt.plot(abscisse, moyenne, 'k--', label='Mean')
            plt.errorbar(abscisse, moyenne, yerr=std, color='red', label='Standard deviation', alpha=0.7)

            plt.title(f'Pixel intensity as a function of distance', fontsize=18)# n°{num_image_verte}')
            plt.xlabel('Distance in pixels', fontsize=18)
            plt.ylabel('Pixel intensity', fontsize=18)
            plt.legend(fontsize=14)

            plt.tight_layout()
            plt.show()

            # On recupere la pente_globale
            pente_globale = linregress(abscisse, moyenne)

            # Détection de l'augmentation autour de l'abscisse 0
            indice_0 = - start_intensite
            pente_locale = linregress(abscisse[indice_0 - 20:indice_0 + 10], moyenne[indice_0 - 20:indice_0 + 10])

            if pente_globale.slope < 0:
                if pente_locale.slope > 0:
                    print("C'est une couronne")  # Augmentation autour de 0
                else:
                    print("C'est une cellule saine")  # Décroissance globale
            else:
                print("On ne peut conclure")  # Pas de classification si non décroissante

            print(f"Pente globale : {pente_globale.slope}")
#           On pourrait dire quand p-value inf à un seuil, -20 par exemple, alors on est sur du résultat
            print(f"Coeff de corrélation : {pente_globale.rvalue}")
            print(f"P-value : {pente_globale.pvalue}")            
            print(f"Pente locale (abscisse -20 a 10): {pente_locale.slope}")
            print(f"Coeff de corrélation : {pente_locale.rvalue}")
            print(f"P-value de la pente locale : {pente_locale.pvalue}")

            num_contour += 1
        
# Essai de tests

#         # Corrélation entre distance et intensité 
#         for i in range(pixels_intensity_perception.shape[0]):
#             corr, _ = pearsonr(abscisse, pixels_intensity_perception[i, :])
#             print(f"Corrélation Pearson pour l'échantillon {i+1}: {corr}")

#         # Régression linéaire entre distance et intensité 
#         X = abscisse.reshape(-1, 1)
#         y = moyenne
#         reg = LinearRegression().fit(X, y)
#         print(f"Régression linéaire: Coefficient = {reg.coef_[0]}, Intercept = {reg.intercept_}")

In [1]:
def contour_binary_fonction(image_bleue, num_image_bleue):
    
    # Plot de l'image originale bleue
    plt.figure(figsize=(12,8))
    plt.subplot(3,3,1)
    plt.imshow(image_bleue)
    plt.title(f'Original blue img°{num_image_bleue}', fontsize=18)
    plt.axis('off')
    
    # On rend noir le fond bleu extracellulaire
    mean_blue = cv2.mean(image_bleue)[2]
#     print(mean_blue)
    hsv_image = cv2.cvtColor(image_bleue, cv2.COLOR_RGB2HSV)
    
    if 10 <= mean_blue < 30:
        lower_bound = np.array([120, 255, 60])
        upper_bound = np.array([120, 255, 255])
        mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
        image_modifie = cv2.bitwise_and(image_bleue, image_bleue, mask=mask)
    elif 30 <= mean_blue < 50:
        lower_bound = np.array([120, 255, 80])
        upper_bound = np.array([120, 255, 255])
        mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
        image_modifie = cv2.bitwise_and(image_bleue, image_bleue, mask=mask)
    elif 50 <= mean_blue < 70:
        lower_bound = np.array([120, 255, 100])
        upper_bound = np.array([120, 255, 255])
        mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
        image_modifie = cv2.bitwise_and(image_bleue, image_bleue, mask=mask)
    elif 70 <= mean_blue < 90:
        lower_bound = np.array([120, 255, 180])
        upper_bound = np.array([120, 255, 255])
        mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
        image_modifie = cv2.bitwise_and(image_bleue, image_bleue, mask=mask)
    elif 90 <= mean_blue < 110:
        lower_bound = np.array([120, 255, 145])
        upper_bound = np.array([120, 255, 255])
        mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
        image_modifie = cv2.bitwise_and(image_bleue, image_bleue, mask=mask)
    else:
        image_modifie = image_bleue
    
    gray_image = cv2.cvtColor(image_modifie, cv2.COLOR_RGB2GRAY) # Image en nuance de gris
    contrast_image = cv2.convertScaleAbs(gray_image, alpha=20) # On augmente le contraste
    blurred_image = cv2.GaussianBlur(contrast_image, (5,5), 0) # On floute
    _, binary_image = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_OTSU) # On met en noir et blanc
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
    closed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel) # On comble les trous
    opened_image = cv2.morphologyEx(closed_image, cv2.MORPH_OPEN, kernel) # On supprime les bruits
    
    # On plot l'image après pré-traitement des données
    plt.subplot(3,3,2)
    plt.imshow(opened_image, cmap='gray')
    plt.title(f'Pre-processed img°{num_image_bleue}', fontsize=18)
    plt.axis('off')
    
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # On recup les contours
    convexe_contoured_image = image_bleue.copy() 
    
    contours_convexe = []
    for contour in contours:
            contour_convexe = cv2.convexHull(contour) # On les rend convexes
            if len(contour_convexe) > 40: # On ne garde que les contours les plus grands
                contours_convexe.append(contour_convexe)
                cv2.drawContours(convexe_contoured_image, [contour_convexe], -1, (255, 255, 0), 2) # Contour convexe
    
    # Plot de l'image bleue avec contours détectés
    plt.subplot(3,3,3)
    plt.imshow(convexe_contoured_image)
    plt.title(f'Contours detected of img°{num_image_bleue}', fontsize=18)
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return contours, convexe_contoured_image, contours_convexe

In [ ]:
def points_image_fonction(contour, image_bleue):
    distance_seuil = 5
    points_contour = []

    for i in range(len(contour)):
        start_point = contour[i]
        end_point = contour[(i + 1) % len(contour)]
        
        distance = np.linalg.norm(end_point - start_point)

        points_contour.append(start_point) 
        if distance > distance_seuil: # Si il y a trop d'écarts entre les points
            points_a_ajouter = int(distance // distance_seuil)

            for j in range(1, points_a_ajouter + 1):
                t = j / (points_a_ajouter + 1)
                point_segment = (1 - t) * start_point + t * end_point
                points_contour.append(point_segment) # On ajoute des points dans le contour pour avoir plus de normales

    points_contour = np.array(points_contour)
    points_contour = points_contour.astype(int)
    
    # On trace le contour et on recupere les cood x et y dans Max_X Max_Y
    contoured_image_points = image_bleue.copy()
    Max_X = []
    Max_Y = []
    
    for point in points_contour:
        x, y = point[0][0], point[0][1] 
        Max_X.append(x)
        Max_Y.append(y)
        cv2.circle(contoured_image_points, (x, y), 3, (255, 255, 0), -1) 
        
    return points_contour, Max_X, Max_Y, contoured_image_points

In [ ]:
def normale_fonction(points_contour, Max_X, Max_Y, normale_length, colors):
    normales = []
    
    # Pour chaque points, on trace la tangente, qui sera la normale
    for a in range(len(points_contour)):
        if a > 0:
            point_prec = points_contour[a - 1]
        else:
            point_prec = points_contour[-1] 

        if a < len(points_contour) - 1:
            point_suiv = points_contour[a + 1] 
        else:
            point_suiv = points_contour[0]

        x_prec, y_prec = point_prec[0][0], point_prec[0][1]
        x_suiv, y_suiv = point_suiv[0][0], point_suiv[0][1]
        tangente = (x_suiv - x_prec, y_suiv - y_prec)

        normale = (tangente[1], -tangente[0])
        norm = np.sqrt(normale[0]**2 + normale[1]**2)
        normale = (normale[0] / norm, normale[1] / norm)
        normales.append(normale)

    return normales

In [ ]:
def all_intensity_fonction(points_contour, Max_X, Max_Y, normale_length, start_intensite, normales, image_verte, num_image_verte, colors):
    
    # Intensité de start_intensité jusqu'à la fin de la normale
    pixels_intensity_perception = np.zeros([len(points_contour),normale_length-start_intensite])
    pixels_x = np.zeros([len(points_contour),normale_length-start_intensite])
    pixels_y = np.zeros([len(points_contour),normale_length-start_intensite])

    image_intensity = image_verte.copy()
    contoured_image_normales = image_verte.copy()
    
    # On crée les couleurs pour les normales
    colors_fonction = colors.copy()
    for c in range(len(points_contour)):
        colors_fonction[c][0] = colors_fonction[c][0] * 255
        colors_fonction[c][1] = colors_fonction[c][1] * 255
        colors_fonction[c][2] = colors_fonction[c][2] * 255
        
    for b in range(len(points_contour)): # Boucle sur les points du contour
        for d in range(start_intensite, normale_length): # Boucle sur les points d'une normale
            X = int(Max_X[b] + normales[b][0] * d)
            Y = int(Max_Y[b] + normales[b][1] * d)

            # Si le point de la normale est dans l'image on la recupere
            if 0 <= X < image_intensity.shape[1] and 0 <= Y < image_intensity.shape[0]:
                indice_positif = d - start_intensite
                pixels_x[b][indice_positif] = X
                pixels_y[b][indice_positif] = Y
                pixel = image_intensity[Y, X]
                pixel_intensity_perception = 0.299 * pixel[0] + 0.587 * pixel[1] + 0.114 * pixel[2] # On récupère l'intensité du pixel
                if pixel_intensity_perception > 24: # 24, il faut mettre le même chiffre dans le calcul de la moyenne ensuite
                    pixels_intensity_perception[b][indice_positif] = pixel_intensity_perception # On garde l'intensité si elle est sup à 24, c'est à dire dans la cellule
                    cv2.circle(contoured_image_normales, (X,Y), radius=1, color=colors_fonction[b], thickness=-1)  # -1 pour un cercle rempli
                else:
                    pixels_intensity_perception[b][indice_positif] = -1
    
    # Tracé des graphiques
    plt.figure(figsize=(12,8))
    plt.subplot(2,2,1)
    plt.imshow(image_verte)
    plt.title(f'Original img°{num_image_verte}', fontsize=18)
    plt.axis('off')

    plt.subplot(2,2,2)
    plt.imshow(contoured_image_normales)
    plt.title(f'Normals of img°{num_image_verte}', fontsize=18)
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
            
    return pixels_intensity_perception, pixels_x, pixels_y, contoured_image_normales

In [ ]:
def mean_std_fonction(pixels_intensity_perception):
    masque = (pixels_intensity_perception < 24)
    matrice_masquee = np.ma.masked_array(pixels_intensity_perception, mask=masque)
    moyenne = np.ma.mean(matrice_masquee, axis=0)
    variance = np.ma.var(matrice_masquee, axis=0)
    std = np.sqrt(variance)
    
    return moyenne, std 